#FlipFeed

###Objetivo:
> Identificar cuando una persona realiza una dominada correctamente

###Metodo:
> Usaremos en este caso, redes neuronales recurrentes para obtener un valor booleano

###Herramientas:

0.   Python
1.   Keras - TensorFLow
2.   Yolo v8 - entrenado con un data set personalizado
3.   OpenCV optical flow






##Yolo v8 y OpenCV Optical Flow
Yolov8 lo utilizamos para etiquetar los objetos en el video, podiendo así extraer las coordenadas en donde se encuentra la persona y el balón.
Acontinuación instalamos todos los paquetes necesarios.

In [ ]:
!pip install torch numpy opencv-python ultralytics supervision==0.1.0 

In [6]:
# Import the necesary libraries
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO

from supervision.draw.color import ColorPalette
from supervision.tools.detections import Detections, BoxAnnotator

In [7]:
import os

cwd = os.getcwd()
files = os.listdir(cwd)
print("Files in %r: %s" % (cwd, files))

Files in '/content': ['.config', 'sample_data']


##OpenCV Optical Flow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#To draw the lines on the frame
def draw_flow(img, flow, step=20):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr

#Capture the video and generate a output with cv2
videoPath = 'dom2.mp4'
input = cv2.VideoCapture(videoPath)
output = cv2.VideoWriter('./Output/output_' + videoPath, cv2.VideoWriter_fourcc(*'mp4v'), 50, (int(input.get(3)),int(input.get(4))))

suc, prev = input.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)

while True:

    #Extract a frame from video and put on it a B&W format 
    suc, img = input.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #Obtain the motion prediction
    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prevgray = gray

    #Show the frame and write frames on output
    imgToShow = draw_flow(gray, flow)
    cv2.imshow('flow', imgToShow)
    output.write(imgToShow)

    key = cv2.waitKey(5)
    if key == ord('q'):
        break

output.release()
input.release()
cv2.destroyAllWindows()